In [ ]:
print("Ready")

## Construct the model

The code below makes a simple ResNet50 model pretrained on ImageNet data.  If desired, replace this with your own model.  Note there is no training step here because the weights are already coming in pre-filled from the Keras library.

In [ ]:
import tensorflow as tf

def get_input_shape(w, h):
  if tf.keras.backend.image_data_format() == 'channels_first':
    input_shape = (3, w, h)
  else:
    input_shape = (w, h, 3)
  return input_shape

image_w = 224
image_h = 224
dropout = 0.5

input_shape = get_input_shape(image_w, image_h)

input_tensor = tf.keras.layers.Input(shape=input_shape)

base_model = tf.keras.applications.ResNet50(
    include_top=True,
    weights="imagenet",
    input_tensor=input_tensor
)

#### Summarize the model and validate that it works

Print a summary of the model.

In [ ]:
print(base_model.summary())

Construct `x`, our NumPy array of two images to be classified by the local model.  For the test images cat & dog.jpg, just find a random picture of a cat and a dog on the Internet, preferably with roughly square aspect ratio.  For cat224 and dog224, use an image editor to size the images to 224*224 pixels.

In [ ]:
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
import numpy as np

img_path1 = 'cat224.jpg'
img1 = image.load_img(img_path1, target_size=(224, 224))
img_path2 = 'dog224.jpg'
img2 = image.load_img(img_path2, target_size=(224, 224))
x = image.img_to_array(img1)
x = np.expand_dims(x, axis=0)
y = image.img_to_array(img2)
y = np.expand_dims(y, axis=0)
x = np.append(x, y, axis=0)
x = preprocess_input(x)

Perform inference on at least one obvious image to make sure classification works as expected.  Here we provide images of a cat and dog at 224*224 pixels.

In [ ]:
preds = base_model.predict(x)
# decode the results into a list of tuples (class, description, probability)
# (one such list for each sample in the batch)
print('Predicted:', decode_predictions(preds, top=3))


## Save the model

Use the Tensorflow `save` function provided in TF2 to very simply save the base model to the desired path.  No Session needed.  The path to `export/Servo/` is required by Amazon SageMaker, and the `0000000001/` is simply a model index so that Tensorflow Serving will automatically serve the correct model version.

In [ ]:
import os

model_save_path = os.path.join("", "export/Servo/0000000001/")
tf.saved_model.save(base_model, model_save_path)

#### Tar & gzip the model for convenient deployment

And because you have to provide a tarred Gzipped file as an argument to a function later ;)

In [ ]:
!tar chvfz test-resnet50.tar.gz export/*

## Re-import the model

Do not bother uploading the SavedModel to S3 yourself.  This will only cause you much grief.  Instead, instantiate a SageMaker Session which will calculate the correct S3 bucket to use and handle everything on its own.

By using the `Model` class instead of `TensorFlowModel` (as demonstrated in https://github.com/aws/amazon-sagemaker-examples/blob/master/sagemaker-python-sdk/tensorflow_serving_container/tensorflow_serving_container.ipynb ), we get support for Python 3 since it's Amazon's understanding that (even still) TensorFlow Serving API library doesn't support Python 3.  This is elaborated on if you read the GitHub issue post https://github.com/aws/sagemaker-python-sdk/issues/912#issuecomment-510226311 .

In [ ]:
import sagemaker
from sagemaker.tensorflow.serving import Model

sagemaker_session = sagemaker.Session()
inputs = sagemaker_session.upload_data(path='test-resnet50.tar.gz', key_prefix='test-resnet50')

sagemaker_model = Model(
    model_data=inputs,
    role='arn:aws:iam::232188586941:role/amazon-sagemaker-practice',
    framework_version='2.2.0'
)


## Deploy the model onto SageMaker

Note: This step takes a while, perhaps 5-10 minutes.  As it completes, you'll see dashes `-` appear in the output beneath the code cell.  These tick off every 30 seconds.  You will see a bang `!` when the deployment is completed.

In [ ]:
predictor = sagemaker_model.deploy(initial_instance_count=1, instance_type='ml.c5.xlarge')

#### Find the endpoint name

This should be the same name for the endpoint as shown on the AWS Console, e.g. `tensorflow-inference-2020-10-27-17-54-06-462`

In [ ]:
predictor.endpoint

## Invoke the SageMaker Model from Scratch

If you didn't bother with any of the other steps above, and want to go straight into calling an existing SageMaker model for inference, then you need to instantiate a Predictor.  (And just to make this even weirder, it seems like this uses SageMaker V1 syntax rather than V2 syntax, but trying to import the libraries and run the code in V2 format fails on my instance.)

In [ ]:
from sagemaker.tensorflow.serving import Predictor

# Note that moving to Predictor V2 will have different defaults for
# serializer & deserializer, which are currently both JSON in this code
predictor = Predictor(
    endpoint_name='tensorflow-inference-2020-10-27-17-54-06-462'
)

### Test the model

Toward the top of this notebook, we defined `x` as an array consisting of two pictures to classify.  Go back and run that if you haven't done it already.  We can use `x` as-is with the SageMaker endpoint.

In [ ]:
# Use x from before, when constructing arrays for inception on the original Keras model loaded in memory
result = predictor.predict(x)

cat_preds = result['predictions'][0]
cat_preds = np.expand_dims(cat_preds, axis=0)
dog_preds = result['predictions'][1]
dog_preds = np.expand_dims(dog_preds, axis=0)
all_preds = np.append(cat_preds, dog_preds, axis=0)
print(all_preds.shape)
decode_predictions(all_preds, top=3)